In [446]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import math

# Leitura instância

In [447]:
def read_dat_file(file_path):
    """"Função para a leitura das instâncias de Mariá"""
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # 1. Lendo quantidade de itens e períodos
    items, periods = map(int, lines[0].split())

    # 2. Lendo número de plantas
    num_plants = int(lines[1].strip())

    # 3. Lendo capacidades das plantas
    capacities = [int(lines[i + 2].strip()) for i in range(num_plants)]
    capacities = np.tile(capacities, (periods, 1)).T  # Repete as capacidades ao longo dos períodos (deixar na forma j, t)

    # 4. Lendo a matriz de produção (tempo de produção, tempo de setup, custo de setup, custo de produção)
    production_data = []
    start_line = 2 + num_plants
    production_time = np.zeros((items, num_plants))  # Inicializar listas para armazenar separadamente os tempos e custos
    setup_time = np.zeros((items, num_plants))
    setup_cost = np.zeros((items, num_plants))
    production_cost = np.zeros((items, num_plants))
    for i in range(num_plants * items):  # Preencher as matrizes com os dados lidos
        plant = i // items  # Determina a planta
        item = i % items    # Determina o item
        # Extrair os dados de cada linha
        prod_time, set_time, set_cost, prod_cost = map(float, lines[start_line + i].split())
        production_time[item, plant] = prod_time  # Preencher as respectivas matrizes
        setup_time[item, plant] = set_time
        setup_cost[item, plant] = set_cost
        production_cost[item, plant] = prod_cost

    # 5. Lendo os custos de inventário
    inventory_costs_line = start_line + num_plants * items
    inventory_costs = list(map(float, lines[inventory_costs_line].split()))  # Lê todos os valores de inventory_costs como uma única lista
    inventory_costs = np.array(inventory_costs).reshape(num_plants, -1)  # Divide a lista de custos de inventário por planta
    inventory_costs = inventory_costs.T  # Deixa na forma (i, j)

    # 6. Lendo a matriz de demanda (12 linhas, 12 colunas)
    demand_matrix = []
    demand_start_line = inventory_costs_line + 1

    # Verifica se a matriz de demanda está na forma padrão
    num_demand_lines = 0
    for i in range(demand_start_line, len(lines)):
        if lines[i].strip():  # Se a linha não estiver vazia, contamos como linha de demanda
            num_demand_lines += 1
        else:
            break  # Interrompe se encontrar uma linha vazia (ou um bloco separado)
    
    # Calcula o multiplicador, caso tenha mais linhas do que períodos
    if num_demand_lines == periods * 2:
        multiplier = 2  # A matriz de demanda tem o dobro de linhas
    else:
        multiplier = 1  # A matriz de demanda tem o número esperado de linhas (T)
    
    # Leitura inicial das demandas
    for i in range(periods * multiplier):  # Lê as linhas de demandas para os períodos
        demands = list(map(int, lines[demand_start_line + i].split()))
        demand_matrix.append(demands)

    # print('Linha 1:', demand_matrix[0])
    # print('Linha 13:', demand_matrix[periods], '\n\n')
    
    # Agora, se multiplier == 2, precisamos combinar as linhas 1-12 com 13-24 corretamente
    if multiplier == 2:
        new_demand_matrix = []
        
        for i in range(periods):  # Para cada linha do primeiro bloco (1-12)
            combined_period_demand = []  # Lista única para armazenar a demanda combinada para o período i
            
            # Quebrar a linha i entre as plantas
            demands_first_part = demand_matrix[i]
            demands_second_part = demand_matrix[i + periods]  # Linha correspondente do segundo bloco
            
            # Calcular o número de elementos por planta
            elements_per_plant_first = len(demands_first_part) // num_plants
            elements_per_plant_second = len(demands_second_part) // num_plants
            
            # Concatenar as demandas para cada planta
            for j in range(num_plants):
                # Particionar a demanda de cada planta
                plant_demand_first = demands_first_part[j*elements_per_plant_first:(j+1)*elements_per_plant_first]
                plant_demand_second = demands_second_part[j*elements_per_plant_second:(j+1)*elements_per_plant_second]
                
                # Concatenar as demandas da planta j (primeiro e segundo blocos) e adicionar diretamente à lista única
                combined_period_demand.extend(plant_demand_first + plant_demand_second)
            
            # Adicionar a demanda ajustada (em formato de lista única) do período à nova matriz de demanda
            new_demand_matrix.append(combined_period_demand)
        
        # Substitui a matriz de demandas pela combinada e particionada por planta
        demand_matrix = new_demand_matrix
    
    # print('Linha 1 output após junção:', demand_matrix[0])

    # Agora vamos dividir os valores de cada linha combinada entre as plantas
    final_demand_matrix = []
    for demands in demand_matrix:
        period_demand = []
        for j in range(num_plants):
            # Divide a demanda combinada por planta, assumindo que cada planta tem o mesmo número de itens
            plant_demand = demands[j*items:(j+1)*items]
            period_demand.append(plant_demand)
        final_demand_matrix.append(period_demand)
    
    # Transpor a matriz de demanda para o formato correto (itens, plantas, períodos)
    final_demand_matrix = np.array(final_demand_matrix)
    # print(final_demand_matrix)
    final_demand_matrix = np.transpose(final_demand_matrix, (2, 1, 0))  # Converte para o formato (itens, plantas, períodos)

    # 7. Lendo os custos de transferência
    transfer_costs = []
    transfer_cost_line = demand_start_line + periods * multiplier
    while transfer_cost_line < len(lines):
        line = lines[transfer_cost_line].strip()  # Verificar se a linha não está vazia antes de tentar ler
        if line:
            transfer_costs.append(float(line))
        transfer_cost_line += 1

    def create_transfer_cost_matrix(transfer_costs, num_plants):  # Criar a matriz de custos de transferência (simétrica)
        transfer_cost_matrix = np.zeros((num_plants, num_plants))  # Inicializar a matriz de zeros
        if len(transfer_costs) == 1:
            transfer_cost = transfer_costs[0]  # Se houver apenas um custo de transferência, aplicar para todos os pares de plantas
            for j in range(num_plants):
                for k in range(j + 1, num_plants):
                    transfer_cost_matrix[j, k] = transfer_cost
                    transfer_cost_matrix[k, j] = transfer_cost
        else:
            idx = 0  # Se houver model_syúltiplos custos, aplicar entre pares de plantas
            for j in range(num_plants):
                for k in range(j + 1, num_plants):
                    transfer_cost_matrix[j, k] = transfer_costs[idx]
                    transfer_cost_matrix[k, j] = transfer_costs[idx]
                    idx += 1
        return transfer_cost_matrix

    transfer_costs = create_transfer_cost_matrix(transfer_costs, num_plants)

    return {"items": items,
            "periods": periods,
            "num_plants": num_plants,
            "capacities": capacities,
            "production_time": production_time,
            "setup_time": setup_time,
            "setup_cost": setup_cost,  
            "production_cost": production_cost,
            "inventory_costs": inventory_costs,
            "demand_matrix": final_demand_matrix,
            "transfer_costs": transfer_costs}

In [448]:
file_path = '../instancias/maria_desiree/NBB01266_2.dat'
data = read_dat_file(file_path)
# display(data)

In [449]:
# Produtos (i)
I = np.array([_ for _ in range(data['items'])])
# Plantas (j)
J = np.array([_ for _ in range(data['num_plants'])])
# Períodos (t)
T = np.array([_ for _ in range(data['periods'])])

# Demanda (i, j, t)
d = np.array(data['demand_matrix'])
# Capacidade (j, t)
cap = np.array(data['capacities'])
# Tempo de produção (i, j)
b = np.array(data['production_time'])
# Tempo de setup (i, j)
f = np.array(data['setup_time'])
# Custo de produção (i, j)
c = np.array(data['production_cost'])
# Custo de setup (i, j)
s = np.array(data['setup_cost'])
# Custo de transporte (j, k)
r = np.array(data['transfer_costs'])
# Custo de estoque (i, j)
h = np.array(data['inventory_costs'])

# Modelo base

In [450]:
model_sy = gp.Model('Lot-sizing Sambasivan and Yahya')

# Quantidade produzida (i, j, t)
X = model_sy.addVars(I, J, T, vtype=GRB.CONTINUOUS, name='X')
# Quantidade estocada (i, j, t)
Q = model_sy.addVars(I, J, T, vtype=GRB.CONTINUOUS, name='Q')
# Quantidade transportada (i, j, k(um outro j), t)
W = model_sy.addVars(I, J, J, T, vtype=GRB.CONTINUOUS, name='W')
# Variável de setup (binária) (i, j, t)
Z = model_sy.addVars(I, J, T, vtype=GRB.BINARY, name='Z')

expr_objetivo = sum(sum(sum(c[i, j] * X[i, j, t] + h[i, j] * Q[i, j, t] + s[i, j] * Z[i, j, t] + 
                            sum(r[j, k] * W[i, j, k, t] for k in J if k != j) for t in T) for j in J) for i in I)
model_sy.setObjective(expr_objetivo, sense=GRB.MINIMIZE)

# Balanço de estoque (revisar comportamento)
# Período inicial
model_sy.addConstrs((Q[i, j, t] == X[i, j, t] - sum(W[i, j, k, t] for k in J if k != j) + sum(W[i, l, j, t] for l in J if l != j) - d[i, j, t] for i in I for j in J for t in T if t == 0),
             name='restricao_balanco_estoque')
# Demais períodos
model_sy.addConstrs((Q[i, j, t] == Q[i, j, t-1] + X[i, j, t] - sum(W[i, j, k, t] for k in J if k != j) + sum(W[i, l, j, t] for l in J if l != j) - d[i, j, t] for i in I for j in J for t in T if t > 0),
             name='restricao_balanco_estoque');

# Restrição que obriga setup (validar o range do r)
# model_sy.addConstrs((X[i, j, t] <= min((cap[j, t] - f[i, j]) / b[i, j], sum(sum(d[i, k, r] for r in range(t, T[-1] + 1)) for k in J)) * Z[i, j, t] for i in I for j in J for t in T)
#              , name='restricao_setup');
model_sy.addConstrs((X[i, j, t] <= sum(sum(d[i, j, b] for b in range(t, len(T))) for j in J) * Z[i, j, t] for i in I for j in J for t in T)
             , name='restricao_setup');

# Restrição de capacidade
model_sy.addConstrs((sum(b[i, j] * X[i, j, t] + f[i, j] * Z[i, j, t] for i in I) <= cap[j, t] for j in J for t in T)
             , name='restricao_capacidade');

In [451]:
# Critérios de parada
model_sy.Params.timelimit = 1800  # Tempo (default = inf)
# model_sy.Params.MIPgap = 0.01  # Gap de otimalidade (default = 0.0001 = 0.01%)
model_sy.Params.OutputFlag = 0
# Otimização
model_sy.optimize()

Set parameter TimeLimit to value 1800


In [452]:
print('UB:', model_sy.ObjVal)
print('LB:', model_sy.ObjBound)

UB: 77368.26444168697
LB: 77366.14526534216


In [453]:
# Armazenamento das soluções
X_sy = X
Q_sy = Q
W_sy = W
Z_sy = Z

# Modelo reformulação

In [464]:
reformulacao = gp.Model('Lot-sizing Reformulation')

In [465]:
# Quantidade produzida (i, j, t, k, u)
X = reformulacao.addVars(I, J, T, J, T, vtype=GRB.CONTINUOUS, name='X')
# Variável de setup (binária) (i, j, t)
Z = reformulacao.addVars(I, J, T, vtype=GRB.BINARY, name='Z')

In [466]:
expr1 = sum(sum(sum(sum(sum(
    X[i, j, t, k, u] * (c[i, j] + min((u - t) * h[i, v] + r[j, v] + r[v, k] for v in J) if u >= t else 0) 
    for u in T) for k in J) for t in T) for j in J) for i in I)
expr2 = sum(sum(sum(s[i, j] * Z[i, j, t] for t in T) for j in J) for i in I)
reformulacao.setObjective(expr1 + expr2, sense=GRB.MINIMIZE)

In [467]:
# Balanço de estoque
reformulacao.addConstrs((sum(sum(X[i, j, t, k, u] for t in range(u + 1)) for j in J) == d[i, k, u] for i in I for k in J for u in T), name='restricao_balanco_estoque');

In [468]:
# Restrição que obriga setup
reformulacao.addConstrs((X[i, j, t, k, u] <= min(math.floor((cap[j, t] - f[i, j]) / b[i, j]), d[i, k, u]) * Z[i, j, t] for i in I for j in J for t in T for k in J for u in T)
             , name='restricao_setup');

In [469]:
# Restrição de capacidade
reformulacao.addConstrs((sum(f[i, j] * Z[i, j, t] + sum(sum(b[i, j] * X[i, j, t, k, u] for u in T) for k in J) for i in I) <= cap[j, t] for j in J for t in T)
             , name='restricao_capacidade');

In [470]:
# FORÇAR SOLUÇÕES DE PRODUÇÃO ANTERIOR NESSA FORMULAÇÃO
reformulacao.addConstrs((sum(sum(X[i, j, t, k, u] for k in J) for u in range(t, T[-1] + 1)) == X_sy[i, j, t].X for i in I for j in J for t in T)
                        , name='force_Xijt_bounds');

In [471]:
# Critérios de parada
reformulacao.Params.timelimit = 1800  # Tempo (default = inf)
# reformulacao.Params.MIPgap = 0.01  # Gap de otimalidade (default = 0.0001 = 0.01%)
reformulacao.Params.OutputFlag = 1
# Otimização
reformulacao.optimize()

Set parameter TimeLimit to value 1800
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 32040 rows, 31536 columns and 127224 nonzeros
Model fingerprint: 0x1a6e4458
Variable types: 31104 continuous, 432 integer (432 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [2e+00, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 31950 rows and 30607 columns
Presolve time: 0.12s
Presolved: 90 rows, 929 columns, 1858 nonzeros
Variable types: 929 continuous, 0 integer (0 binary)

Root relaxation: objective 7.736826e+04, 32 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap |

In [472]:
print('UB:', reformulacao.ObjVal)
print('LB:', reformulacao.ObjBound)

UB: 77368.26444168696
LB: 77368.26444168696
